written by Yodai Takei (ytakei@caltech.edu), Feburary 2022

In [2]:
# Our numerical workhorses
import numpy as np
import pandas as pd
import re
from collections import defaultdict, Counter
import math
import scipy.optimize
import os
import glob

# Plotting modules
import matplotlib.pyplot as plt
import matplotlib.patches

# Package to perform PCA
import sklearn.datasets
import sklearn.decomposition

# This is to enable inline displays for the purposes of the tutorial
%matplotlib inline

# This enables SVG graphics inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Seaborn makes plots look nice
import seaborn as sns


rc = {'lines.linewidth': 2, 'axes.labelsize': 18,  'axes.titlesize': 18, 'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('ticks')

# Suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
data_path_2 = "I:/OneDrive - California Institute of Technology/Long Cai - 1/100k/LC1-100k-000-reference-analysis/output/"
file_name_2 = "LC1-100k-000-006-mouse-whole-genome-25kb-blocks-annotated.csv"
df_ref = pd.read_csv(data_path_2+file_name_2)
df_ref = df_ref[['name','chrom','start','end','compartment']]
df_ref.head()

,name,chrom,start,end,compartment
0,chr1-1,chr1,3000000,3025000,B
1,chr1-2,chr1,3025000,3050000,B
2,chr1-3,chr1,3050000,3075000,B
3,chr1-4,chr1,3075000,3100000,B
4,chr1-5,chr1,3100000,3125000,B


In [4]:
chrom_list = list(df_ref['chrom'].unique())
chrom_list

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chrX']

In [5]:
def hull_intersection(points,hull):
    p_eq = hull.equations
    normal, dist = p_eq[:,:-1],p_eq[:,-1]
    alpha = -dist/np.dot(normal,points.T)
    return np.multiply(np.min(alpha[alpha>0]),points)

In [6]:
def surf_scdist(df_f,col1='n_rad_score',col2='n_per_dist(um)'):
    
    df_f1 = df_f.copy(deep=True)
    
    df_f1['x_um'] = df_f1['x_um'] - df_f1['x_um'].mean()
    df_f1['y_um'] = df_f1['y_um'] - df_f1['y_um'].mean()
    df_f1['z_um'] = df_f1['z_um'] - df_f1['z_um'].mean()
    
    x = df_f1['x_um']
    y = df_f1['y_um']
    z = df_f1['z_um']
    
    points = np.array(df_f1[['x_um','y_um','z_um']])
    hull = scipy.spatial.ConvexHull(points)
    
    closest_point = []
    for i in range(len(df_f1)):
        closest_point.append(list(hull_intersection(points[i],hull)))
        
    df_cp = pd.DataFrame(closest_point)

    x1 = df_cp[0]
    y1 = df_cp[1]
    z1 = df_cp[2]

    sqrt1 = np.array(np.sqrt(x**2 + y**2 + z**2))
    sqrt2 = np.array(np.sqrt(x1**2 + y1**2 + z1**2))

    # normalized distance from center
    norm = sqrt1/sqrt2 

    # absolute distance from lamin or nuclear periphery
    # in this case, this is identical to np.array(np.sqrt((x-x1)**2 + (y-y1)**2 + (z-z1)**2))
    lamin_um = sqrt2 - sqrt1  

    df_norm = pd.DataFrame([norm,lamin_um]).transpose()
    
    df_norm.columns = [col1,col2]
    
    return df_norm

# computed radial score from nuclear center as well as distance (um) from nuclear periphery, using all loci for brain

In [10]:
path = 'I:/OneDrive - California Institute of Technology/hpc-data/2022-02-25-updated-results/'

max_p = [3,4]
file_name = ['Brain_rep1_pos_','Brain_rep2_pos_']

thresh = 100 # minimum number of spots per cell

for r in range(2):
    for i in range(max_p[r]):
        max_dcd = pd.read_csv(path+file_name[r]+str(i)+'.csv')
        #max_dcd = max_dcd[max_dcd['dbscan_ldp_nbr_allele']!=-1].reset_index(drop=True)
        max_dcd['x_um'] = max_dcd['x']*0.103
        max_dcd['y_um'] = max_dcd['y']*0.103
        max_dcd['z_um'] = max_dcd['z']*0.250
        
        df_nr = []
        for j in range(max_dcd['cellID'].max()):
            print(r+1, i, j+1)
            df_f2 = max_dcd[max_dcd['cellID']==j+1].reset_index(drop=True)
            # compute radial scores from nuclear center as well as distance (um) from nuclear periphery
            if len(df_f2)> thresh:
                df_norm = surf_scdist(df_f2,col1='n_rad_score',col2='n_per_dist(um)')
                df_nrc = pd.concat([df_f2,df_norm],axis=1)
    
                if len(df_nr) == 0:
                    df_nr = df_nrc.copy(deep=True)
                else:
                    df_nr = pd.concat([df_nr,df_nrc],axis=0).reset_index(drop=True) 
                
        df_nr.to_csv('output/'+file_name[r]+str(i)+'_radial_scores_wofilter.csv')

1 0 1
1 0 2
1 0 3
1 0 4
1 0 5
1 0 6
1 0 7
1 0 8
1 0 9
1 0 10
1 0 11
1 0 12
1 0 13
1 0 14
1 0 15
1 0 16
1 0 17
1 0 18
1 0 19
1 0 20
1 0 21
1 0 22
1 0 23
1 0 24
1 0 25
1 0 26
1 0 27
1 0 28
1 0 29
1 0 30
1 0 31
1 0 32
1 0 33
1 0 34
1 0 35
1 0 36
1 0 37
1 0 38
1 0 39
1 0 40
1 0 41
1 0 42
1 0 43
1 0 44
1 0 45
1 0 46
1 0 47
1 0 48
1 0 49
1 0 50
1 0 51
1 0 52
1 0 53
1 0 54
1 0 55
1 0 56
1 0 57
1 0 58
1 0 59
1 0 60
1 0 61
1 0 62
1 0 63
1 0 64
1 0 65
1 0 66
1 0 67
1 0 68
1 0 69
1 0 70
1 0 71
1 0 72
1 0 73
1 0 74
1 0 75
1 0 76
1 0 77
1 0 78
1 0 79
1 0 80
1 0 81
1 0 82
1 0 83
1 0 84
1 0 85
1 0 86
1 0 87
1 0 88
1 0 89
1 0 90
1 0 91
1 0 92
1 0 93
1 0 94
1 0 95
1 0 96
1 0 97
1 0 98
1 0 99
1 0 100
1 0 101
1 0 102
1 0 103
1 0 104
1 0 105
1 0 106
1 0 107
1 0 108
1 0 109
1 0 110
1 0 111
1 0 112
1 0 113
1 0 114
1 0 115
1 0 116
1 0 117
1 0 118
1 0 119
1 0 120
1 0 121
1 0 122
1 0 123
1 0 124
1 0 125
1 0 126
1 0 127
1 0 128
1 0 129
1 0 130
1 0 131
1 0 132
1 0 133
1 0 134
1 0 135
1 0 136
1 0 137
1 0 138
1 0 